# Molecular dynamics (MD):

## Langevin dynamics:

In physics, Langevin dynamics is an approach to the mathematical modelling of the dynamics of molecular systems. 

The approach is characterized by the use of simplified models while accounting for omitted degrees of freedom by the use of stochastic differential equations. Langevin dynamics simulations are a kind of Monte Carlo simulation.

### Langevin thermostat:

Langevin dynamics allows temperature to be controlled like with a thermostat, thus approximating the canonical ensemble.

A canonical ensemble is the statistical ensemble that represents the possible states of a mechanical system in thermal equilibrium with a heat bath at a fixed temperature.


A canonical ensemble does not evolve over time, despite the fact that the underlying system is in constant motion. This is because the ensemble is only a function of a conserved quantity of the system (energy).


The Langevin thermostat maintains the temperature through a modification of Newton's equations of motion:

$$
\dot{r_i} = \frac{p_i}{m_i} \qquad
\dot{p_i} = F_i - {\gamma}_i\,p_i + f_i,
$$

where $F_i$ is the force acting on atom $i$ due to the interaction potential, $\gamma_i$ is a friction coefficient, and $f_i$ is a random force simulating the random kicks by the damping of particles between each other due to friction. The random numbers are chosen from a Gaussian distribution with the following variance

$$
\sigma_i^2 = \frac{2\,m_i\,{\gamma}_i\,k_B\,T}{{\Delta}t}
$$

with $\Delta t$ being the time-step used in the MD to integrate the equations of motion. Obviously, Langevin dynamics is identical to the classical Hamiltonian in the limit of vanishing $\gamma$.


### Visualisation:

We will use OVITO, get it from here:

https://www.ovito.org/



#### References:

https://en.wikipedia.org/wiki/Langevin_dynamics

https://en.wikipedia.org/wiki/Canonical_ensemble

https://www.vasp.at/wiki/index.php/Langevin_thermostat


#### Code reference:

Andrew Abi-Mansour: https://github.com/Comp-science-engineering/Tutorials/tree/master/MolecularDynamics

The referecen script implements the Langevin dynamics by:

- Updating positions based on current velocities.

- Updating velocities based on the total force, which includes:

 - A friction term proportional to momentum (velocity * mass / relaxation time).

 - A random force drawn from a Gaussian distribution with a variance that depends on particle mass, temperature, friction coefficient, and time step.

-  The parameter relax in the code functions as the inverse of the friction coefficient $\gamma_i$, or more precisely, the relaxation time $\tau$ where $\gamma_i=\frac{1}{\tau}$. This is a common way to parameterize the Langevin thermostat.


In [1]:
# !/usr/bin/python
# -------------------------------------------------------------------------
#   A simple molecular dynamics solver that simulates the motion
#   of non-interacting particles in the canonical ensemble using
#   a Langevin thermostat by Andrew Abi-Mansour
# -------------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt

# Define global physical constants
from scipy.constants import Avogadro, Boltzmann

def wallHitCheck(pos, vels, box):
    """ This function enforces reflective boundary conditions.
    All particles that hit a wall  have their velocity updated
    in the opposite direction.
    @pos: atomic positions (ndarray)
    @vels: atomic velocity (ndarray, updated if collisions detected)
    @box: simulation box size (tuple)
    """
    ndims = len(box)

    for i in range(ndims):
        vels[((pos[:,i] <= box[i][0]) | (pos[:,i] >= box[i][1])),i] *= -1

def integrate(pos, vels, forces, mass,  dt):
    """ A simple forward Euler integrator that moves the system in time 
    @pos: atomic positions (ndarray, updated)
    @vels: atomic velocity (ndarray, updated)
    """
    pos += vels * dt
    vels += forces * dt / mass[np.newaxis].T
    
def computeForce(mass, vels, temp, relax, dt):
    """ Computes the Langevin force for all particles
    @mass: particle mass (ndarray)
    @vels: particle velocities (ndarray)
    @temp: temperature (float)
    @relax: thermostat constant (float)
    @dt: simulation timestep (float)
    returns forces (ndarray)
    """
    natoms, ndims = vels.shape

    sigma = np.sqrt(2.0 * mass * temp * Boltzmann / (relax * dt))
    noise = np.random.randn(natoms, ndims) * sigma[np.newaxis].T

    force = - (vels * mass[np.newaxis].T) / relax + noise
    
    return force

def run(**args):
    """ This is the main function that solves Langevin's equations for
    a system of natoms usinga forward Euler scheme, and returns an output
    list that stores the time and the temperture.
    @natoms (int): number of particles
    @temp (float): temperature (in Kelvin)
    @mass (float): particle mass (in Kg)
    @relax (float): relaxation constant (in seconds)
    @dt (float): simulation timestep (s)
    @nsteps (int): total number of steps the solver performs
    @box (tuple): simulation box size (in meters) of size dimensions x 2
    e.g. box = ((-1e-9, 1e-9), (-1e-9, 1e-9)) defines a 2D square
    @ofname (string): filename to write output to
    @freq (int): write output every 'freq' steps
    @[radius]: particle radius (for visualization)
    Returns a list (of size nsteps x 2) containing the time and temperature.
    """
    natoms, box, dt, temp = args['natoms'], args['box'], args['dt'], args['temp']
    mass, relax, nsteps   = args['mass'], args['relax'], args['steps']
    ofname, freq, radius = args['ofname'], args['freq'], args['radius']
    
    dim = len(box)
    pos = np.random.rand(natoms,dim)

    for i in range(dim):
        pos[:,i] = box[i][0] + (box[i][1] -  box[i][0]) * pos[:,i]

    vels = np.random.rand(natoms,dim)
    mass = np.ones(natoms) * mass / Avogadro
    radius = np.ones(natoms) * radius
    step = 0

    output = []

    while step <= nsteps:

        step += 1

        # Compute all forces
        forces = computeForce(mass, vels, temp, relax, dt)

        # Move the system in time
        integrate(pos, vels, forces, mass, dt)

        # Check if any particle has collided with the wall
        wallHitCheck(pos,vels,box)

        # Compute output (temperature)
        ins_temp = np.sum(np.dot(mass, (vels - vels.mean(axis=0))**2)) / (Boltzmann * dim * natoms)
        output.append([step * dt, ins_temp])
        
        if not step%freq:
            #dump.writeOutput(ofname, natoms, step, box, radius=radius, pos=pos, v=vels)
            writeOutput(ofname, natoms, step, box, radius=radius, pos=pos, v=vels)
    return np.array(output)


def writeOutput(filename, natoms, timestep, box, **data):
    """ Writes the output (in dump format) """
    
    axis = ('x', 'y', 'z')
    
    with open(filename, 'a') as fp:
        
        fp.write('ITEM: TIMESTEP\n')
        fp.write('{}\n'.format(timestep))
        
        fp.write('ITEM: NUMBER OF ATOMS\n')
        fp.write('{}\n'.format(natoms))
        
        fp.write('ITEM: BOX BOUNDS' + ' f' * len(box) + '\n')
        for box_bounds in box:
            fp.write('{} {}\n'.format(*box_bounds))

        for i in range(len(axis) - len(box)):
            fp.write('0 0\n')
            
        keys = list(data.keys())
        
        for key in keys:
            isMatrix = len(data[key].shape) > 1
            
            if isMatrix:
                _, nCols = data[key].shape
                
                for i in range(nCols):
                    if key == 'pos':
                        data['{}'.format(axis[i])] = data[key][:,i]
                    else:
                        data['{}_{}'.format(key,axis[i])] = data[key][:,i]
                        
                del data[key]
                
        keys = data.keys()
        
        fp.write('ITEM: ATOMS' + (' {}' * len(data)).format(*data) + '\n')
        
        output = []
        for key in keys:
            output = np.hstack((output, data[key]))
          
        if len(output):
            np.savetxt(fp, output.reshape((natoms, len(data)), order='F'))

In [2]:
# Run code wih desired parameters

N_atoms = 10

if __name__ == '__main__':

    params = {
        'natoms': N_atoms,
        'temp': 300,
        'mass': 0.001,
        'radius': 120e-12,
        'relax': 1e-13,
        'dt': 1e-15,
        'steps': 10000,
        'freq': 100,
        'box': ((0, 1e-8), (0, 1e-8), (0, 1e-8)),
        'ofname': 'traj-hydrogen-3D-{}.dump'.format(N_atoms)
        }

    output = run(**params)

    plt.figure(figsize=(6,5))
    plt.plot(output[:,0] * 1e12, output[:,1])
    plt.xlabel('Time (ps)')
    plt.ylabel('Temp (K)')
    plt.savefig("./temperature-{}.png".format(N_atoms))
    plt.close()
    